In [2]:
import os

# Set the base path to the current working directory
base_path = os.getcwd()

print(f"Models and scalers will be saved in: {base_path}")

Models and scalers will be saved in: C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction


In [13]:
import requests

BINANCE_BASE_URL = "https://api.binance.com/api/v3"

def fetch_available_pairs():
    response = requests.get(f"{BINANCE_BASE_URL}/exchangeInfo")
    symbols = response.json().get("symbols", [])
    return {s["symbol"] for s in symbols if s["status"] == "TRADING"}

def check_combinations(crypto_symbols, base_symbols, time_frames):
    available_pairs = fetch_available_pairs()
    unsupported_combinations = []
    supported = []

    for crypto_symbol, base_symbol in product(crypto_symbols, base_symbols):
        symbol = f"{crypto_symbol}{base_symbol}"
        if symbol not in available_pairs:
            unsupported_combinations.append(symbol)
        else:
            supported.append(symbol)

    # Check time frames if needed
    # Note: This is a placeholder as time frames are generally supported across all pairs
    supported_time_frames = {"1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"}
    for time_frame in time_frames:
        if time_frame not in supported_time_frames:
            print(f"Unsupported time frame: {time_frame}")

    return unsupported_combinations, supported

# Example usage
CRYPTO_SYMBOLS = ["BTC", "ETH", "BNB", "XRP", "ADA", "SOL", "DOGE", "SHIB", "LTC", "TRX", "AVAX", "ATOM", "LINK"] 
BASE_SYMBOLS = ["USDT", "USDC", "EUR", "JPY", "TRY"]

TIME_FRAMES = ["15m", "30m", "1h", "4h", "1d", "1w"]

unsupported, supported = check_combinations(CRYPTO_SYMBOLS, BASE_SYMBOLS, TIME_FRAMES)
print("Unsupported combinations:", unsupported)
print(f"{len(unsupported)}")

print("Supported combinations:", supported)
print(f"{len(supported)}")


Unsupported combinations: ['DOTJPY', 'LTCJPY', 'TRXJPY', 'AVAXJPY', 'ATOMJPY', 'LINKJPY']
6
Supported combinations: ['BTCUSDT', 'BTCUSDC', 'BTCEUR', 'BTCJPY', 'BTCTRY', 'ETHUSDT', 'ETHUSDC', 'ETHEUR', 'ETHJPY', 'ETHTRY', 'BNBUSDT', 'BNBUSDC', 'BNBEUR', 'BNBJPY', 'BNBTRY', 'XRPUSDT', 'XRPUSDC', 'XRPEUR', 'XRPJPY', 'XRPTRY', 'ADAUSDT', 'ADAUSDC', 'ADAEUR', 'ADAJPY', 'ADATRY', 'SOLUSDT', 'SOLUSDC', 'SOLEUR', 'SOLJPY', 'SOLTRY', 'DOGEUSDT', 'DOGEUSDC', 'DOGEEUR', 'DOGEJPY', 'DOGETRY', 'DOTUSDT', 'DOTUSDC', 'DOTEUR', 'DOTTRY', 'SHIBUSDT', 'SHIBUSDC', 'SHIBEUR', 'SHIBJPY', 'SHIBTRY', 'LTCUSDT', 'LTCUSDC', 'LTCEUR', 'LTCTRY', 'TRXUSDT', 'TRXUSDC', 'TRXEUR', 'TRXTRY', 'AVAXUSDT', 'AVAXUSDC', 'AVAXEUR', 'AVAXTRY', 'ATOMUSDT', 'ATOMUSDC', 'ATOMEUR', 'ATOMTRY', 'LINKUSDT', 'LINKUSDC', 'LINKEUR', 'LINKTRY']
64


In [ ]:
import requests

def fetch_top_50_symbols():
    BINANCE_BASE_URL = "https://api.binance.com/api/v3"
    response = requests.get(f"{BINANCE_BASE_URL}/ticker/24hr")
    data = response.json()

    # Sort by volume and get the top 50
    sorted_data = sorted(data, key=lambda x: float(x['quoteVolume']), reverse=True)
    top_50 = sorted_data[:50]

    # Extract symbols
    top_50_symbols = [item['symbol'] for item in top_50]
    return top_50_symbols

top_50_symbols = fetch_top_50_symbols()
print("Top 50 Symbols by Volume:")
for symbol in top_50_symbols:
    print(symbol)



In [16]:
from itertools import product
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import requests
import pickle

# Define your symbols and time frames
# CRYPTO_SYMBOLS = ["BTC", "ETH", "BNB", "XRP", "ADA", "SOL", "DOGE", "DOT", "MATIC", "SHIB", "LTC", "TRX", "AVAX", "ATOM", "LINK"]
# BASE_SYMBOLS = ["USDT", "BUSD", "USDC", "BTC", "ETH", "BNB", "EUR", "GBP", "AUD", "JPY", "TRY", "RUB", "CAD", "CNY", "KRW"]
# TIME_FRAMES = ["15m", "30m", "1h", "2h", "4h", "1d", "3d", "1w", "1M"]

CRYPTO_SYMBOLS = ["BTC", "ETH", "BNB", "XRP", "ADA", "SOL", "DOGE", "SHIB", "LTC", "TRX", "AVAX", "ATOM", "LINK"] 
BASE_SYMBOLS = ["USDT", "USDC", "EUR", "JPY", "TRY"]

TIME_FRAMES = ["15m", "30m", "1h", "4h", "1d", "1w"]

# Base path for saving models and scalers
base_path = base_path = os.getcwd()

# Function to fetch candlestick data
def fetch_candlestick_data(symbol, interval, limit=1000):
    BINANCE_BASE_URL = "https://api.binance.com/api/v3"
    url = f"{BINANCE_BASE_URL}/klines?symbol={symbol}&interval={interval}&limit={limit}"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data, columns=[
        'OpenTime', 'Open', 'High', 'Low', 'Close', 'Volume',
        'CloseTime', 'QuoteAssetVolume', 'Trades', 'TakerBuyBase', 'TakerBuyQuote', 'Ignore'])
    return df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)

# Function to train the model
def train_model(symbol, interval, base_path):
    model_path = os.path.join(base_path, f"{symbol}_{interval}_model.keras")
    scaler_path = os.path.join(base_path, f"{symbol}_{interval}_scaler.pkl")

    data = fetch_candlestick_data(symbol, interval)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

    x_train, y_train = [], []
    for i in range(60, len(scaled_data)):
        x_train.append(scaled_data[i-60:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    model = Sequential()
    model.add(Input(shape=(x_train.shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1)
    model_checkpoint = ModelCheckpoint(model_path, save_best_only=True, monitor='loss', verbose=1)

    model.fit(x_train, y_train, batch_size=32, epochs=50, callbacks=[early_stopping, model_checkpoint])

    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)

    return model, scaler

# Iterate over all combinations
for crypto_symbol, base_symbol, time_frame in product(CRYPTO_SYMBOLS, BASE_SYMBOLS, TIME_FRAMES):
    symbol = f"{crypto_symbol}{base_symbol}"
    print(f"Training model for {symbol} at {time_frame} interval")
    
    # Train the model
    try:
        train_model(symbol, time_frame, base_path)
    except Exception as e:
        print(f"Failed to train model for {symbol} at {time_frame}: {e}")


Training model for BTCUSDT at 15m interval
Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.1124
Epoch 1: loss improved from inf to 0.04725, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_15m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 74ms/step - loss: 0.1103
Epoch 2/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0080
Epoch 2: loss improved from 0.04725 to 0.00641, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_15m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0079
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.0045
Epoch 3: loss improved from 0.00641 to 0.00439, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_15m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: 0.0045
Epoch 4/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0044
Epoch 4: loss improved from 0.00439 to 0.00406, saving model to C:\Use

Epoch 29: early stopping
Training model for BTCUSDT at 30m interval
Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0800
Epoch 1: loss improved from inf to 0.03546, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_30m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - loss: 0.0785
Epoch 2/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0080
Epoch 2: loss improved from 0.03546 to 0.00694, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_30m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.0079
Epoch 3/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0052
Epoch 3: loss improved from 0.00694 to 0.00498, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_30m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 0.0051
Epoch 4/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0052
Epoch 4: loss did not improve from 0.00498
30

29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0038
Epoch 23: loss did not improve from 0.00353
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 0.0038
Epoch 24/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0034
Epoch 24: loss improved from 0.00353 to 0.00339, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_1h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.0034
Epoch 25/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0034
Epoch 25: loss did not improve from 0.00339
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0034
Epoch 26/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0035
Epoch 26: loss did not improve from 0.00339
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 0.0035
Epoch 27/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0034
Epoch 27: loss did not improve from 0.00339
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - loss: 0.0034
Epoch 28/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0027
Epoch

Epoch 20/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0016
Epoch 20: loss improved from 0.00200 to 0.00172, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_4h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - loss: 0.0016
Epoch 21/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0015
Epoch 21: loss improved from 0.00172 to 0.00167, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_4h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - loss: 0.0015
Epoch 22/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0022
Epoch 22: loss did not improve from 0.00167
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.0022
Epoch 23/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0020
Epoch 23: loss did not improve from 0.00167
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.0020
Epoch 24/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0017
Epoch 24: loss did not improve from 0.00167
30/30

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.0042
Epoch 3: loss improved from 0.00505 to 0.00378, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDC_15m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 0.0042
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0036
Epoch 4: loss improved from 0.00378 to 0.00367, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDC_15m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - loss: 0.0036
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.0034
Epoch 5: loss did not improve from 0.00367
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0034
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.0038
Epoch 6: loss did not improve from 0.00367
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0038
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0038
Epoch 7: loss improved from 0.00367 to 0.00363, saving model to C:\

...

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 0.0096
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0087
Epoch 8: loss did not improve from 0.00991
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0089
Epoch 9/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0080
Epoch 9: loss improved from 0.00991 to 0.00764, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCEUR_1w_model.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 0.0080
Epoch 10/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.0082 
Epoch 10: loss did not improve from 0.00764
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - loss: 0.0086
Epoch 11/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0104
Epoch 11: loss did not improve from 0.00764
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0102
Epoch 12/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.0077
Epoch 12: loss did not improve from 0.00764
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.0080
Epoch 13/50
7/7 ━━━━━━━━━━━━━━━

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0046
Epoch 39/50
6/7 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0039
Epoch 39: loss did not improve from 0.00349
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0040
Epoch 40/50
6/7 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0040
Epoch 40: loss did not improve from 0.00349
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - loss: 0.0039
Epoch 41/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0039
Epoch 41: loss did not improve from 0.00349
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 0.0039
Epoch 41: early stopping
Training model for BTCJPY at 15m interval
Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0672
Epoch 1: loss improved from inf to 0.02862, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCJPY_15m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 80ms/step - loss: 0.0659
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0050
Epoch 2: loss improved from 0.02862 to 0.00431, saving mod

Epoch 13/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0034
Epoch 13: loss improved from 0.00360 to 0.00344, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCJPY_1h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - loss: 0.0034
Epoch 14/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.0034
Epoch 14: loss improved from 0.00344 to 0.00315, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCJPY_1h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - loss: 0.0034
Epoch 15/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.0032
Epoch 15: loss did not improve from 0.00315
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - loss: 0.0033
Epoch 16/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0030
Epoch 16: loss did not improve from 0.00315
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - loss: 0.0030
Epoch 17/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0032
Epoch 17: loss did not improve from 0.00315
30/30 ━

30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 0.0018
Epoch 20/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0021
Epoch 20: loss did not improve from 0.00195
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 0.0021
Epoch 21/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 0.0019
Epoch 21: loss improved from 0.00195 to 0.00189, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCJPY_4h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 111ms/step - loss: 0.0019
Epoch 22/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.0018
Epoch 22: loss improved from 0.00189 to 0.00182, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCJPY_4h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 120ms/step - loss: 0.0018
Epoch 23/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 0.0017
Epoch 23: loss improved from 0.00182 to 0.00172, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCJPY_4h_model.keras
30/3

KeyboardInterrupt: 

In [2]:
from itertools import product
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import requests
import pickle
from datetime import datetime, timedelta
from celery import Celery

# Set up Celery app
# celery_app = Celery('model_training', broker='redis://localhost:6379/0')

# Define your symbols and time frames
CRYPTO_SYMBOLS = ["BTC", "ETH", "BNB", "XRP", "ADA", "SOL", "DOGE", "SHIB", " ", "TRX", "AVAX", "ATOM", "LINK"]
BASE_SYMBOLS = ["USDT", "USDC", "EUR", "JPY", "TRY"]
TIME_FRAMES = ["15m", "30m", "1h", "4h", "1d", "1w"]

# Base path for saving models and scalers
base_path = os.getcwd()

# Function to fetch candlestick data with dynamic date ranges
def fetch_candlestick_data(symbol, interval, start_time, end_time):
    BINANCE_BASE_URL = "https://api.binance.com/api/v3/klines"
    limit = 1000
    df = pd.DataFrame()

    while start_time < end_time:
        url = f"{BINANCE_BASE_URL}?symbol={symbol}&interval={interval}&limit={limit}&startTime={int(start_time.timestamp() * 1000)}"
        response = requests.get(url)
        data = response.json()

        if not data:
            break

        temp_df = pd.DataFrame(data, columns=[
            'OpenTime', 'Open', 'High', 'Low', 'Close', 'Volume',
            'CloseTime', 'QuoteAssetVolume', 'Trades', 'TakerBuyBase', 'TakerBuyQuote', 'Ignore'])
        df = pd.concat([df, temp_df], ignore_index=True)

        start_time = datetime.fromtimestamp(temp_df['CloseTime'].iloc[-1] / 1000)

    return df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)

# Function to prepare data
def prepare_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

    x_train, y_train = [], []
    for i in range(60, len(scaled_data)):
        x_train.append(scaled_data[i-60:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    return x_train, y_train, scaler

# Function to train the model
def train_model(symbol, interval, base_path):
    model_path = os.path.join(base_path, f"{symbol}_{interval}_model.keras")
    scaler_path = os.path.join(base_path, f"{symbol}_{interval}_scaler.pkl")

    # Check if model and scaler exist
    if os.path.exists(model_path) and os.path.exists(scaler_path):
        print(f"Model and scaler already exist for {symbol} at {interval}. Skipping training.")
        return

    # Fetch data for the past 60 days
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=60)
    data = fetch_candlestick_data(symbol, interval, start_time, end_time)

    # Prepare data
    x_train, y_train, scaler = prepare_data(data)

    # Build GRU model
    model = Sequential()
    model.add(Input(shape=(x_train.shape[1], 1)))
    model.add(GRU(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Callbacks
    early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1)
    model_checkpoint = ModelCheckpoint(model_path, save_best_only=True, monitor='loss', verbose=1)

    # Train the model
    model.fit(x_train, y_train, batch_size=32, epochs=50, callbacks=[early_stopping, model_checkpoint])

    # Save scaler
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)

    print(f"Model and scaler saved for {symbol} at {interval}.")
    
    
# Iterate over all combinations
for crypto_symbol, base_symbol, time_frame in product(CRYPTO_SYMBOLS, BASE_SYMBOLS, TIME_FRAMES):
    symbol = f"{crypto_symbol}{base_symbol}"
    print(f"Training model for {symbol} at {time_frame} interval")
    
    # Train the model
    try:
        train_model(symbol, time_frame, base_path)
    except Exception as e:
        print(f"Failed to train model for {symbol} at {time_frame}: {e}")

# Celery task to retrain models
# @celery_app.task
# def retrain_models():
#     for crypto_symbol, base_symbol, time_frame in product(CRYPTO_SYMBOLS, BASE_SYMBOLS, TIME_FRAMES):
#         symbol = f"{crypto_symbol}{base_symbol}"
#         print(f"Retraining model for {symbol} at {time_frame} interval")

#         try:
#             train_model(symbol, time_frame, base_path)
#         except Exception as e:
#             print(f"Failed to train model for {symbol} at {time_frame}: {e}")

# Entry point for manual execution
# if __name__ == "__main__":
#     retrain_models()


Training model for BTCUSDT at 15m interval
Model and scaler already exist for BTCUSDT at 15m. Skipping training.
Training model for BTCUSDT at 30m interval
Model and scaler already exist for BTCUSDT at 30m. Skipping training.
Training model for BTCUSDT at 1h interval
Model and scaler already exist for BTCUSDT at 1h. Skipping training.
Training model for BTCUSDT at 4h interval
Model and scaler already exist for BTCUSDT at 4h. Skipping training.
Training model for BTCUSDT at 1d interval
Failed to train model for BTCUSDT at 1d: tuple index out of range
Training model for BTCUSDT at 1w interval
Failed to train model for BTCUSDT at 1w: tuple index out of range
Training model for BTCUSDC at 15m interval
Model and scaler already exist for BTCUSDC at 15m. Skipping training.
Training model for BTCUSDC at 30m interval
Model and scaler already exist for BTCUSDC at 30m. Skipping training.
Training model for BTCUSDC at 1h interval
Model and scaler already exist for BTCUSDC at 1h. Skipping training.

Failed to train model for BNBEUR at 1d: tuple index out of range
Training model for BNBEUR at 1w interval
Failed to train model for BNBEUR at 1w: tuple index out of range
Training model for BNBJPY at 15m interval
Model and scaler already exist for BNBJPY at 15m. Skipping training.
Training model for BNBJPY at 30m interval
Model and scaler already exist for BNBJPY at 30m. Skipping training.
Training model for BNBJPY at 1h interval
Model and scaler already exist for BNBJPY at 1h. Skipping training.
Training model for BNBJPY at 4h interval
Model and scaler already exist for BNBJPY at 4h. Skipping training.
Training model for BNBJPY at 1d interval
Failed to train model for BNBJPY at 1d: tuple index out of range
Training model for BNBJPY at 1w interval
Failed to train model for BNBJPY at 1w: tuple index out of range
Training model for BNBTRY at 15m interval
Model and scaler already exist for BNBTRY at 15m. Skipping training.
Training model for BNBTRY at 30m interval
Model and scaler already

Failed to train model for SOLUSDT at 1d: tuple index out of range
Training model for SOLUSDT at 1w interval
Failed to train model for SOLUSDT at 1w: tuple index out of range
Training model for SOLUSDC at 15m interval
Model and scaler already exist for SOLUSDC at 15m. Skipping training.
Training model for SOLUSDC at 30m interval
Model and scaler already exist for SOLUSDC at 30m. Skipping training.
Training model for SOLUSDC at 1h interval
Model and scaler already exist for SOLUSDC at 1h. Skipping training.
Training model for SOLUSDC at 4h interval
Model and scaler already exist for SOLUSDC at 4h. Skipping training.
Training model for SOLUSDC at 1d interval
Failed to train model for SOLUSDC at 1d: tuple index out of range
Training model for SOLUSDC at 1w interval
Failed to train model for SOLUSDC at 1w: tuple index out of range
Training model for SOLEUR at 15m interval
Model and scaler already exist for SOLEUR at 15m. Skipping training.
Training model for SOLEUR at 30m interval
Model and

Failed to train model for SHIBJPY at 1d: tuple index out of range
Training model for SHIBJPY at 1w interval
Failed to train model for SHIBJPY at 1w: tuple index out of range
Training model for SHIBTRY at 15m interval
Model and scaler already exist for SHIBTRY at 15m. Skipping training.
Training model for SHIBTRY at 30m interval
Model and scaler already exist for SHIBTRY at 30m. Skipping training.
Training model for SHIBTRY at 1h interval
Model and scaler already exist for SHIBTRY at 1h. Skipping training.
Training model for SHIBTRY at 4h interval
Model and scaler already exist for SHIBTRY at 4h. Skipping training.
Training model for SHIBTRY at 1d interval
Failed to train model for SHIBTRY at 1d: tuple index out of range
Training model for SHIBTRY at 1w interval
Failed to train model for SHIBTRY at 1w: tuple index out of range
Training model for  USDT at 15m interval
Failed to train model for  USDT at 15m: single positional indexer is out-of-bounds
Training model for  USDT at 30m interv

Failed to train model for AVAXUSDT at 1d: tuple index out of range
Training model for AVAXUSDT at 1w interval
Failed to train model for AVAXUSDT at 1w: tuple index out of range
Training model for AVAXUSDC at 15m interval
Model and scaler already exist for AVAXUSDC at 15m. Skipping training.
Training model for AVAXUSDC at 30m interval
Model and scaler already exist for AVAXUSDC at 30m. Skipping training.
Training model for AVAXUSDC at 1h interval
Model and scaler already exist for AVAXUSDC at 1h. Skipping training.
Training model for AVAXUSDC at 4h interval
Model and scaler already exist for AVAXUSDC at 4h. Skipping training.
Training model for AVAXUSDC at 1d interval
Failed to train model for AVAXUSDC at 1d: tuple index out of range
Training model for AVAXUSDC at 1w interval
Failed to train model for AVAXUSDC at 1w: tuple index out of range
Training model for AVAXEUR at 15m interval
Model and scaler already exist for AVAXEUR at 15m. Skipping training.
Training model for AVAXEUR at 30m 

Failed to train model for LINKEUR at 1d: tuple index out of range
Training model for LINKEUR at 1w interval
Failed to train model for LINKEUR at 1w: tuple index out of range
Training model for LINKJPY at 15m interval
Failed to train model for LINKJPY at 15m: single positional indexer is out-of-bounds
Training model for LINKJPY at 30m interval
Failed to train model for LINKJPY at 30m: single positional indexer is out-of-bounds
Training model for LINKJPY at 1h interval
Failed to train model for LINKJPY at 1h: single positional indexer is out-of-bounds
Training model for LINKJPY at 4h interval
Failed to train model for LINKJPY at 4h: single positional indexer is out-of-bounds
Training model for LINKJPY at 1d interval
Failed to train model for LINKJPY at 1d: single positional indexer is out-of-bounds
Training model for LINKJPY at 1w interval
Failed to train model for LINKJPY at 1w: single positional indexer is out-of-bounds
Training model for LINKTRY at 15m interval
Model and scaler alread

In [16]:
from itertools import product
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import requests
import pickle
from datetime import datetime, timedelta

# Define your symbols and time frames
CRYPTO_SYMBOLS = ["BTC", "ETH", "BNB", "XRP", "ADA", "SOL", "DOGE", "SHIB", "TRX", "AVAX", "ATOM", "LINK"]
BASE_SYMBOLS = ["USDT", "USDC", "EUR", "JPY", "TRY"]
TIME_FRAMES = ["1d"]

# Base path for saving models and scalers
base_path = os.getcwd()

# Function to fetch candlestick data with dynamic date ranges
def fetch_candlestick_data(symbol, interval, start_time, end_time):
    BINANCE_BASE_URL = "https://api.binance.com/api/v3/klines"
    limit = 1000
    df = pd.DataFrame()

    while start_time < end_time:
        url = f"{BINANCE_BASE_URL}?symbol={symbol}&interval={interval}&limit={limit}&startTime={int(start_time.timestamp() * 1000)}"
        response = requests.get(url)
        data = response.json()

        if not data:
            print(f"No data returned for {symbol} at {interval}.")
            break

        temp_df = pd.DataFrame(data, columns=[
            'OpenTime', 'Open', 'High', 'Low', 'Close', 'Volume',
            'CloseTime', 'QuoteAssetVolume', 'Trades', 'TakerBuyBase', 'TakerBuyQuote', 'Ignore'])

        if temp_df.empty:
            print(f"No data available for {symbol} at {interval}.")
            break

        df = pd.concat([df, temp_df], ignore_index=True)
        start_time = datetime.fromtimestamp(temp_df['CloseTime'].iloc[-1] / 1000)

    if df.empty:
        raise ValueError(f"No data fetched for {symbol} at {interval}.")

    return df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)


def prepare_data(data):
    # Check if there are enough data points
    if len(data) < 60:
        raise ValueError(f"Not enough data points ({len(data)}) for training. Need at least 60.")

    # Print the shape of the data for debugging
    print(f"Data shape in prepare_data: {data.shape}")

    # Verify Data Columns
    print("Column names:", data.columns)
    if data['Close'].isnull().any():
        print("Warning: Missing values detected in 'Close' column.")
    print("Sample 'Close' data:", data['Close'].head())

    # Scale the 'Close' prices
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

    # Print the length and contents of scaled_data for debugging
    # print(f"length: {len(scaled_data)}, Scaled data: {scaled_data}")

    x_train, y_train = [], []
    if len(scaled_data) > 60:
        for i in range(60, len(scaled_data)):
            x_train.append(scaled_data[i-60:i, 0])
            y_train.append(scaled_data[i, 0])

    # Check if x_train and y_train have been populated
    if not x_train or not y_train:
        print("x_train or y_train is empty. Check the data length and preparation logic.")
        return None, None, None

    # Convert lists to numpy arrays
    x_train, y_train = np.array(x_train), np.array(y_train)

    # Reshape x_train for the GRU model
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    return x_train, y_train, scaler



# Function to train the model
def train_model(symbol, interval, base_path):
    model_path = os.path.join(base_path, f"{symbol}_{interval}_model.keras")
    scaler_path = os.path.join(base_path, f"{symbol}_{interval}_scaler.pkl")

    # Check if model and scaler exist
    if os.path.exists(model_path) and os.path.exists(scaler_path):
        print(f"Model and scaler already exist for {symbol} at {interval}. Skipping training.")
        return

    # Fetch data for the past 60 days
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=120)
    data = fetch_candlestick_data(symbol, interval, start_time, end_time)

    # Prepare data
    x_train, y_train, scaler = prepare_data(data)

    # Build GRU model
    model = Sequential()
    model.add(Input(shape=(x_train.shape[1], 1)))
    model.add(GRU(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Callbacks
    early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1)
    model_checkpoint = ModelCheckpoint(model_path, save_best_only=True, monitor='loss', verbose=1)

    # Train the model
    model.fit(x_train, y_train, batch_size=32, epochs=50, callbacks=[early_stopping, model_checkpoint])

    # Save scaler
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)

    print(f"Model and scaler saved for {symbol} at {interval}.")

# Iterate over all combinations
for crypto_symbol, base_symbol, time_frame in product(CRYPTO_SYMBOLS, BASE_SYMBOLS, TIME_FRAMES):
    symbol = f"{crypto_symbol}{base_symbol}"
    print(f"Training model for {symbol} at {time_frame} interval")
    
    # Train the model
    try:
        train_model(symbol, time_frame, base_path)
    except Exception as e:
        print(f"Failed to train model for {symbol} at {time_frame}: {e}")


Training model for BTCUSDT at 1d interval
Data shape in prepare_data: (120, 5)
Column names: Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Sample 'Close' data: 0    56180.00
1    53962.97
2    54160.86
3    54869.95
4    57042.00
Name: Close, dtype: float64
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step - loss: 0.6038
Epoch 1: loss improved from inf to 0.58369, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_1d_model.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 17s 605ms/step - loss: 0.5971
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 0.3647
Epoch 2: loss improved from 0.58369 to 0.33869, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_1d_model.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 181ms/step - loss: 0.3560
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - loss: 0.1712
Epoch 3: loss improved from 0.33869 to 0.14963, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\cryp

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - loss: 0.0239
Epoch 13: loss did not improve from 0.01177
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - loss: 0.0247
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 0.0214
Epoch 14: loss did not improve from 0.01177
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - loss: 0.0209
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.0200
Epoch 15: loss did not improve from 0.01177
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - loss: 0.0198
Epoch 15: early stopping
Model and scaler saved for BTCUSDC at 1d.
Training model for BTCEUR at 1d interval
Data shape in prepare_data: (120, 5)
Column names: Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Sample 'Close' data: 0    50576.57
1    48908.49
2    48994.27
3    49591.03
4    51724.70
Name: Close, dtype: float64
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.6978
Epoch 1: loss improved from inf to 0.67007, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\cry

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step - loss: 0.0881
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 0.0721
Epoch 9: loss did not improve from 0.01585
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - loss: 0.0710
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 0.0395
Epoch 10: loss did not improve from 0.01585
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 0.0381
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0198 
Epoch 11: loss did not improve from 0.01585
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 0.0199
Epoch 11: early stopping
Model and scaler saved for BTCJPY at 1d.
Training model for BTCTRY at 1d interval
Data shape in prepare_data: (120, 5)
Column names: Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Sample 'Close' data: 0    1916925.0
1    1856273.0
2    1857282.0
3    1875344.0
4    1945841.0
Name: Close, dtype: float64
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - loss: 0.6267
Epoch 1: loss improved from inf to 0.59543, 

Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - loss: 0.0188
Epoch 15: loss did not improve from 0.01649
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step - loss: 0.0184
Epoch 15: early stopping
Model and scaler saved for ETHUSDT at 1d.
Training model for ETHUSDC at 1d interval
Data shape in prepare_data: (120, 5)
Column names: Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Sample 'Close' data: 0    2368.13
1    2224.84
2    2273.39
3    2297.39
4    2360.20
Name: Close, dtype: float64
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.4199
Epoch 1: loss improved from inf to 0.39672, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\ETHUSDC_1d_model.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 178ms/step - loss: 0.4122
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.2534 
Epoch 2: loss improved from 0.39672 to 0.23553, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\ETHUSDC_1d_model.keras
2/2 ━━━━━━

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step - loss: 0.0120
Epoch 17/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 0.0125 
Epoch 17: loss did not improve from 0.01316
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 0.0129
Epoch 18/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0205 
Epoch 18: loss did not improve from 0.01316
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0198
Epoch 19/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 0.0145 
Epoch 19: loss did not improve from 0.01316
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 0.0143
Epoch 20/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 0.0158 
Epoch 20: loss did not improve from 0.01316
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - loss: 0.0152
Epoch 21/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.0152
Epoch 21: loss did not improve from 0.01316
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.0148
Epoch 21: early stopping
Model and scaler saved for ETHEUR at 1d.
Training model for ETHJPY at 1d interval
Data shape in prepare

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - loss: 0.0176
Epoch 20: loss did not improve from 0.01364
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step - loss: 0.0172
Epoch 21/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - loss: 0.0146
Epoch 21: loss did not improve from 0.01364
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step - loss: 0.0152
Epoch 22/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - loss: 0.0169
Epoch 22: loss did not improve from 0.01364
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 218ms/step - loss: 0.0178
Epoch 23/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - loss: 0.0162
Epoch 23: loss did not improve from 0.01364
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step - loss: 0.0168
Epoch 24/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 297ms/step - loss: 0.0195
Epoch 24: loss did not improve from 0.01364
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 307ms/step - loss: 0.0192
Epoch 24: early stopping
Model and scaler saved for AVAXEUR at 1d.
Training model for AVAXJPY at 1d interval
No data available for AVAXJPY at 1d.
Failed to train model for AVAXJPY at 1d: No da

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 0.0215
Epoch 12: loss did not improve from 0.01689
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - loss: 0.0213
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - loss: 0.0234
Epoch 13: loss did not improve from 0.01689
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - loss: 0.0228
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 0.0224
Epoch 14: loss did not improve from 0.01689
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: 0.0236
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 0.0176
Epoch 15: loss did not improve from 0.01689
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 0.0175
Epoch 16/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 0.0232
Epoch 16: loss did not improve from 0.01689
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 0.0234
Epoch 16: early stopping
Model and scaler saved for ATOMUSDT at 1d.
Training model for ATOMUSDC at 1d interval
Data shape in prepare_data: (120, 5)
Column names: Index(['Open', 'High', 'Low', 

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - loss: 0.0175
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.0191
Epoch 10: loss did not improve from 0.01576
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - loss: 0.0188
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - loss: 0.0223
Epoch 11: loss did not improve from 0.01576
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - loss: 0.0224
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 0.0268
Epoch 12: loss did not improve from 0.01576
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - loss: 0.0270
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - loss: 0.0249
Epoch 13: loss did not improve from 0.01576
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 0.0241
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 0.0261
Epoch 14: loss did not improve from 0.01576
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 0.0249
Epoch 14: early stopping
Model and scaler saved for ATOMEUR at 1d.
Training model for ATOMJPY at 1d interval
No data available

Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.0164
Epoch 12: loss improved from 0.01792 to 0.01512, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\LINKUSDT_1d_model.keras
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - loss: 0.0160
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 0.0247
Epoch 13: loss did not improve from 0.01512
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - loss: 0.0236
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 0.0261
Epoch 14: loss did not improve from 0.01512
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 0.0253
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 0.0214
Epoch 15: loss did not improve from 0.01512
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - loss: 0.0220
Epoch 16/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 0.0224
Epoch 16: loss did not improve from 0.01512
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - loss: 0.0224
Epoch 17/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - loss: 0.0232
Epo

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - loss: 0.0265
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 0.0299
Epoch 14: loss did not improve from 0.01761
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - loss: 0.0287
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.0236
Epoch 15: loss did not improve from 0.01761
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.0239
Epoch 15: early stopping
Model and scaler saved for LINKEUR at 1d.
Training model for LINKJPY at 1d interval
No data available for LINKJPY at 1d.
Failed to train model for LINKJPY at 1d: No data fetched for LINKJPY at 1d.
Training model for LINKTRY at 1d interval
Data shape in prepare_data: (120, 5)
Column names: Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')
Sample 'Close' data: 0    342.2
1    327.8
2    343.3
3    353.1
4    358.9
Name: Close, dtype: float64
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.4239
Epoch 1: loss improved from inf to 0.42150, saving model to C:\Users\